In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.optimizers import adam

Using TensorFlow backend.


In [17]:
#Open the file and read contents
raw_text=open('Alice.txt').read()
raw_text=raw_text.lower()

#Mapping the characters to integers
chars=sorted(list(set(raw_text)))
char_to_int=dict((c,i) for i, c in enumerate(chars))
int_to_char=dict((i,c) for i, c in enumerate(chars))

#Print the number of unique characters
print("Toal %d characters found. "%(len(chars)))
print(chars[1:10])

#Print total words in the vocabulary
print("Total %d words found in book" %len(raw_text))

Toal 45 characters found. 
[' ', '!', '"', "'", '(', ')', '*', ',', '-']
Total 144408 words found in book


In [18]:
#Length of sequence to learn from is defined below
seq_length=75
X=[]
y=[]

for i in range(0, len(raw_text)-seq_length):
    seq_in=raw_text[i:i+seq_length]
    seq_out=raw_text[i+seq_length]
    X.append([char_to_int[char] for char in seq_in])
    y.append(char_to_int[seq_out])
    
#Here we have mapped 74 words sequence in X and the 75 character which is our target variable in Y

#total such patterns created
print("Total %d patterns are created"%len(X))

Total 144333 patterns are created


In [19]:
#First we must transform the list of input sequences into the form [samples, time steps, features] 
#expected by an LSTM network.
dataX=X
n_patterns=len(y)
X=np.reshape(X, (n_patterns, seq_length, 1))

# one hot encode the output variable
y = np_utils.to_categorical(y)

print (y[5:12])

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.

In [ ]:
#Creating the LSTM Model

model=Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# define the checkpoint
filepath="weights/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# fit the model
model.fit(X, y, nb_epoch=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
144333/144333 [==============================] - 3061s - loss: 2.6125  
Epoch 2/20
144333/144333 [==============================] - 3182s - loss: 2.2800  
Epoch 3/20
144333/144333 [==============================] - 4013s - loss: 2.0897  
Epoch 4/20
144333/144333 [==============================] - 3642s - loss: 1.9639  
Epoch 5/20
144333/144333 [==============================] - 3591s - loss: 1.8752  
Epoch 6/20
144333/144333 [==============================] - 3612s - loss: 1.8044  
Epoch 7/20
144333/144333 [==============================] - 3643s - loss: 1.7387  
Epoch 8/20
144333/144333 [==============================] - 3648s - loss: 1.6836  
Epoch 9/20
144333/144333 [==============================] - 3646s - loss: 1.6389  
Epoch 10/20
144333/144333 [==============================] - 3656s - loss: 1.6002  
Epoch 11/20
144333/144333 [==============================] - 3659s - loss: 1.5650  
Epoch 12/20
144333/144333 [==============================] - 3673s - loss: 1.5389  
E

In [23]:
# load the network weights
model=Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

filename = "weights/weights-improvement-16-1.4225.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [25]:
import sys
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Pattern is {}".format(pattern) )
print ("Seed:")
print ("{}" .format(int_to_char[value[0]] for value in pattern))
# generate characters
for i in range(500):
	x = np.reshape(pattern, (1, len(pattern), 1))
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")


Pattern is [8, 1, 38, 26, 33, 39, 25, 26, 10, 0, 26, 23, 1, 41, 19, 37, 1, 19, 32, 1, 33, 30, 22, 1, 21, 36, 19, 20, 8, 1, 26, 23, 1, 41, 19, 37, 10, 4, 0, 0, 4, 27, 1, 32, 23, 40, 23, 36, 1, 41, 23, 32, 38, 1, 38, 33, 1, 26, 27, 31, 8, 4, 1, 38, 26, 23, 1, 31, 33, 21, 29, 1, 38, 39, 36]
Seed:
<generator object <genexpr> at 0x7f6f96ac2db0>
tle said to herself, 'i don't know
the things thing i shink you mike,' said the mock turtle.

'i don't know it was at all the sabbit,' said the mock turtle.

'i don't know it was at all the sabbit,' said the mock turtle.

'i don't know it was at all the sabbit,' said the mock turtle.

'i don't know it was at all the sabbit,' said the mock turtle.

'i don't know it was at all the sabbit,' said the mock turtle.

'i don't know it was at all the sabbit,' said the mock turtle.

'i don't know it was a
Done.
